In [1]:
import csv
import pandas as pd
import os
import glob
import xarray as xr
import rasterio
import pathlib

12-4
- this is a bit clunky but i think notebook needs to go as follows:
    -make list of RGI shp files
    - make list of paths for each rgi shape file. this ends at {glaicername}_DIR, similar to original filebuild fn input
    - MAIN FUNCTION
        -takes path input
        -makes paths that will go in gdal rasterize command (for shape file and output netcdf)
        -makes a rasterio object for each glacier
        -uses that to get array shape and bbox of each glaicer, this will also go in rasterize command
        -writes command to rasterize 
        -writes into .sh file in new location
        -there will be 1 .sh file per glaicer! need ot figure out where to put
   

In [2]:
#make list of RGI names

everest_study_glaciers_full = os.listdir('/uufs/chpc.utah.edu/common/home/u1269862/written_files/everest_glaciers')
everest_study_glaciers_full

everest_study_glaciers = []
for item in range(len(everest_study_glaciers_full)):
    if (everest_study_glaciers_full[item][-3:] == 'shp'):
        everest_study_glaciers.append(everest_study_glaciers_full[item])
    else:
        pass

In [60]:
everest_study_glaciers

['RGI60-15.03366.shp',
 'RGI60-15.03401.shp',
 'RGI60-15.03417.shp',
 'RGI60-15.03422.shp',
 'RGI60-15.03428.shp',
 'RGI60-15.03435.shp',
 'RGI60-15.03448.shp',
 'RGI60-15.03473.shp',
 'RGI60-15.03474.shp',
 'RGI60-15.03619.shp',
 'RGI60-15.03733.shp',
 'RGI60-15.03734.shp',
 'RGI60-15.03742.shp',
 'RGI60-15.03743.shp',
 'RGI60-15.03744.shp',
 'RGI60-15.03765.shp',
 'RGI60-15.09650.shp',
 'RGI60-15.09675.shp',
 'RGI60-15.09729.shp',
 'RGI60-15.09745.shp',
 'RGI60-15.09760.shp',
 'RGI60-15.09771.shp',
 'RGI60-15.09781.shp',
 'RGI60-15.09792.shp',
 'RGI60-15.09801.shp',
 'RGI60-15.09802.shp',
 'RGI60-15.09803.shp',
 'RGI60-15.09805.shp',
 'RGI60-15.09811.shp',
 'RGI60-15.09839.shp',
 'RGI60-15.09858.shp',
 'RGI60-15.09862.shp',
 'RGI60-15.09868.shp',
 'RGI60-15.09870.shp',
 'RGI60-15.09871.shp',
 'RGI60-15.09876.shp',
 'RGI60-15.09921.shp',
 'RGI60-15.09965.shp',
 'RGI60-15.09991.shp',
 'RGI60-15.10055.shp',
 'RGI60-15.10056.shp',
 'RGI60-15.10070.shp',
 'RGI60-15.10076.shp',
 'RGI60-15.

In [4]:
glacier_path_inputs = []

for glacier in range(len(everest_study_glaciers)):
    
    glac_name = everest_study_glaciers[glacier][:-4]
    glacier_path_inputs.append(f"'/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/{glac_name}_DIR'")
    
    

In [5]:
print(glacier_path_inputs[0])

'/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/RGI60-15.03366_DIR'


In [6]:
#orig_TEST = glacier_bbox_dims('/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/RGI60-15.09745_DIR')


In [123]:
def glacier_bbox_dims(glacier_path_list, everest_study_glaciers):
    
    output_ls = []
    golive_full_scene_dir = '/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_SCENE2/'
    
    for glacier in range(len(glacier_path_list)):
        
        command = ""
        
        #set names to be used w/ string formatting for paths
        fullname = everest_study_glaciers[glacier]
        print(fullname, 'fullname')
        print('')
        glaciername = glacier_path_list[glacier][73:-5]
        shortname = glacier_path_list[glacier][83:-5]
        #print(glaciername)
        
        
        #this is where the new mask .nc file will be written to
        glacier_mask_path = f"'/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/{glaciername}_DIR/{glaciername}_mask/{glaciername}_mask.nc'"
        
        #this is the path to the xr object that needs to be made to get shape, bbox
        vx_path = f"/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/{glaciername}_DIR/{glaciername}_clip_vxmask/"

        #path to shapefile to be rasterized
        shp_path = f"'/uufs/chpc.utah.edu/common/home/u1269862/written_files/everest_glaciers/{fullname}'"
        print('')
        print('SHP PATH', shp_path)
        print('')

        #build xr object
        files_full = sorted(os.listdir(golive_full_scene_dir))
        files_vx = sorted(os.listdir(vx_path))
        vx_ls = []   
        for file in range(len(files_vx)):
            vx_ls.append(rasterio.open(vx_path + files_vx[0]))
                
        #get pixel dims
        ncols = vx_ls[0].shape[1]
        nrows = vx_ls[0].shape[0]
    
        #output order is ROWS, COLS
        pixel_dims = (nrows, ncols)
    
        #get bbox
        xmin = vx_ls[0].bounds[0]
        ymin = vx_ls[0].bounds[1]
        xmax = vx_ls[0].bounds[2]
        ymax = vx_ls[0].bounds[3]
        bbox = (xmin, ymin, xmax, ymax)
        
        tuple1=tuple(pixel_dims)
        tuple2=tuple(bbox)
        output_ls.append((tuple1, tuple2))
        
        command += f"gdal_rasterize -burn 1 -te {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} -tr 300 300 -ts {pixel_dims[1]} {pixel_dims[0]} -of netcdf {shp_path} {glacier_mask_path}"
    
        #write command to .sh file
        cmd_file = f"/uufs/chpc.utah.edu/common/home/u1269862/gdal_commands/DEC_rasterizing_shapefiles/{glaciername}_rasterize.sh"
        f = open(os.path.join(cmd_file), 'w')

        f.write(command)
        f.close()
    
    return output_ls
    #return 


In [124]:
os.chdir('/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/RGI60-15.03366_DIR/RGI60-15.03366_clip_vxmask')

In [125]:
pwd

'/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/RGI60-15.03366_DIR/RGI60-15.03366_clip_vxmask'

In [126]:
orig_TEST = glacier_bbox_dims(glacier_path_inputs, everest_study_glaciers)

RGI60-15.03366.shp fullname


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/everest_glaciers/RGI60-15.03366.shp'

RGI60-15.03401.shp fullname


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/everest_glaciers/RGI60-15.03401.shp'

RGI60-15.03417.shp fullname


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/everest_glaciers/RGI60-15.03417.shp'

RGI60-15.03422.shp fullname


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/everest_glaciers/RGI60-15.03422.shp'

RGI60-15.03428.shp fullname


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/everest_glaciers/RGI60-15.03428.shp'

RGI60-15.03435.shp fullname


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/everest_glaciers/RGI60-15.03435.shp'

RGI60-15.03448.shp fullname


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/everest_glaciers/RGI60-15.03448.shp'

RGI60-15.03473.shp fullname


SHP PATH '/uufs/chpc.utah.edu/common/ho